This notebook allows to load the posterior used to compute summary statistics of galaxy surveys (in 3D_galaxies.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import linalg

from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM

from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.coordinates import concatenate
from astropy import units as u

from matplotlib.patches import Ellipse
from scipy.spatial.transform import Rotation

import treecorr
import random

import torch
from torch.distributions import Uniform

from sbi.utils import BoxUniform
from sbi.analysis import pairplot
from sbi.inference import NPE, SNPE, simulate_for_sbi, infer
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
    prepare_for_sbi
)

import corner

/opt/homebrew/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Number of galaxies considered

nbre_galaxies = 250_000

### Model : 3D gaussian population of galaxies

- Parameters : $\theta = \{\mu_B, \mu_C, \sigma_B, \sigma_C, r\}$

In [3]:
def population_3D (mu_B, mu_C, sigma_B, sigma_C, r, nb_galaxies) : 
    
    # EIGENVALUES
    A=1
    
    # Random gaussian draw to obtain the axis lengths B,C (= eigenvalues)
    BC2 = np.empty((0, 2))
    while len(BC2) < nb_galaxies:
        BC = np.random.multivariate_normal(mean=[mu_B, mu_C], cov=[[sigma_B**2, r*sigma_B*sigma_C], [r*sigma_B*sigma_C, sigma_C**2]], size=nb_galaxies)
    
        # Priors : B/A<1 ; C/A<1 ; C<B ; C>0 ; B>0
        mask = (BC[:,0]/A<=1) & (BC[:,1]/ A<=1) & (BC[:,0] >= BC[:,1]) & (BC[:,0]>0) & (BC[:,1]>0)
        BC_valid = BC[mask]

        BC2 = np.concatenate((BC2, BC_valid), axis=0)
    
    BC2 = BC2[:nb_galaxies]
    B, C = BC2[:, 0], BC2[:, 1]

    A=np.ones(nb_galaxies)

    evls = np.array([A,B,C])**2 ; evls=np.transpose(evls) # eigenvalues



    # EIGENVECTORS

    # Random orientation angles (quaternions)
    rand_quat = np.random.randn(nb_galaxies,4) ; rand_quat /= np.linalg.norm(rand_quat, axis=1, keepdims=True)
    rotation = Rotation.from_quat(rand_quat) ; euler_angles = rotation.as_euler('ZYX', degrees=True)
    euler_angles_rad = euler_angles*np.pi/180
    psi, theta, phi = euler_angles_rad[:,0], euler_angles_rad[:,1], euler_angles_rad[:,2]

    
    # Rotation matrix that relates the eigenvectors (X,Y,Z) of the galaxies to a frame of reference (x,y,z)
    D = np.zeros((nb_galaxies, 3, 3))

    D[:, 0, 0] = np.cos(theta) * np.cos(psi)
    D[:, 0, 1] = -np.cos(phi) * np.sin(psi) + np.sin(phi) * np.sin(theta) * np.cos(psi)
    D[:, 0, 2] = np.sin(phi) * np.sin(psi) + np.cos(phi) * np.sin(theta) * np.cos(psi)

    D[:, 1, 0] = np.cos(theta) * np.sin(psi)
    D[:, 1, 1] = np.cos(phi) * np.cos(psi) + np.sin(phi) * np.sin(theta) * np.sin(psi)
    D[:, 1, 2] = -np.sin(phi) * np.cos(psi) + np.cos(phi) * np.sin(theta) * np.sin(psi)

    D[:, 2, 0] = -np.sin(theta)
    D[:, 2, 1] = np.sin(phi) * np.cos(theta)
    D[:, 2, 2] = np.cos(phi) * np.cos(theta)

    
    evc0 = np.asarray([[1,0,0],[0,1,0],[0,0,1]]) # Frame of reference (x,y,z)
    evcl = np.einsum('ijk,kl->ijl', D, evc0) # eigenvectors (X,Y,Z)

    return evcl, evls

### Priors for the parameters

In [4]:
class CustomPrior(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        # Define uniform priors for the parameters
        self.prior_uniform = BoxUniform(
            low=torch.tensor( [0.0,  0., 0, 0, -0.8]), # for (mu_B, mu_C, sigma_B, sigma_C, r)
            high=torch.tensor([1.0,  1., 1, 1, 0.8])
        )
    
    def log_prob(self, x):
        # Check the constraint mu_B > mu_C
        muB_minus_muC = x[..., 1] - x[..., 0]  # assuming mu_B is at index 0 and mu_C at index 1
        mask = ( muB_minus_muC > 0 )
        return torch.where(mask, self.prior_uniform.log_prob(x), torch.tensor(-float('inf')))
    

    def sample(self, sample_shape=torch.Size()):
        if len(sample_shape) == 0:
            sample_shape = torch.Size([1])  # Default to a single sample if sample_shape is empty
            
        samples = []
        while len(samples) < sample_shape[0]:
            sample = self.prior_uniform.sample((1,))
            if ( -sample[..., 0] + sample[..., 1]).lt(0):  # this checks if it is less than zero, so I switched it around
                samples.append(sample)
                
        return torch.cat(samples, dim=0)

# Instantiate the custom prior
prior = CustomPrior()

In [7]:
# Function required for further work

def e_complex(a,b,r):
        abs_e = (1-(b/a)) / (1+(b/a))
        e1 = abs_e*np.cos(2*r)
        e2 = abs_e*np.sin(2*r)
        return e1, e2

### Simulator to project the 3D galaxies in 2D

In [6]:
# SIMULATOR (for simulation-based inference): projection of the 3D galaxies in 2D along the line-of-sight ('y')
# Output = summary statistics = P(e)


def simulator(theta, # The parameters of the 3D model (mu_B, mu_C, sigma_B, sigma_C, r)
               nb_galaxies=nbre_galaxies, # The number of galaxies
               p_axis='y', # The direction of projection (here 'y' denotes by convention the direction of the line-of-sight)
               A=1, # The major axis of the galaxies, fixed to 1 in the analysis
               e_bins=np.linspace(0,1,50) # Bins for the histogram of e_counts (output)
               ):


    mu_B, mu_C, sigma_B, sigma_C, r = theta
    

    #Eigenvectors and eigenvalues
    evcl, evls = population_3D (mu_B, mu_C, sigma_B, sigma_C, r, nb_galaxies)
    

    # Projection 3D => 2D
    if p_axis=='x': # Projection perpendicular to the LOS
        K = np.sum(evcl[:,:,0][:,:,None]*(evcl/evls[:,None]), axis=1)
        r = evcl[:,:,2] - evcl[:,:,0] * K[:,2][:,None] / K[:,0][:,None]
        s = evcl[:,:,1] - evcl[:,:,0] * K[:,1][:,None] / K[:,0][:,None] 

    if p_axis=='y': # Projection along the LOS
        K = np.sum(evcl[:,:,1][:,:,None] * (evcl/evls[:,None]), axis=1)
        r = evcl[:,:,0] - evcl[:,:,1] * K[:,0][:,None] / K[:,1][:,None]
        s = evcl[:,:,2] - evcl[:,:,1] * K[:,2][:,None] / K[:,1][:,None]


    # Coefficients A,B,C
    A1 = np.sum(r**2 / evls, axis=1)
    B1 = np.sum(2*r*s / evls, axis=1)
    C1 = np.sum(s**2 / evls, axis=1)


    # Axis a_p,b_p and orientation angle r_p of the projected galaxy
    r_p = np.pi / 2 + np.arctan2(B1,A1-C1)/2
    a_p = 1/np.sqrt((A1+C1)/2 + (A1-C1)/(2*np.cos(2*r_p)))
    b_p = 1/np.sqrt(A1+C1-(1/a_p**2))


    # Projected ellipticity
    e1, e2 = e_complex(a_p, b_p, r_p) ; e = [e1,e2] ; e=np.array(e)


    # Final output = summary statistics = P(e)
    e_counts,_ = np.histogram(np.sqrt(e[0,:]**2+e[1,:]**2),bins=e_bins)
    
    
    return e_counts/nb_galaxies

### Running the simulations

In [ ]:
# Running the simulations, training of the neural network and estimation of the posterior (NPE = Neural Posterior Estimation) 

posterior = infer(simulator, prior, method = 'NPE', num_simulations = 60000, num_workers = 10)

### Saving the estimated posterior

In [ ]:
torch.save(posterior, 'posterior.pt')